In [22]:
import numpy as np 
import h5py

In [23]:
def read_geometry_local(file_name):
    #Opens and reads in lines of the magn_geometry_XXXX file.
    f = open(file_name,'r')
    file_raw = f.read()
    file_lines = file_raw.split('\n')

    #Initializes parameters dictionary and populates.
    parameters = {}
    l = 1
    while '/' not in file_lines[l]:
        if len(file_lines[l]) == 0:
            l += 1
            continue
        lsplit = file_lines[l].split('=')
        if lsplit[0].strip() == 'gridpoints':
            parameters[lsplit[0].strip()] = int(float(lsplit[1].strip()))
        elif lsplit[0].strip() == 'magn_geometry':
            parameters[lsplit[0].strip()] = lsplit[1].strip()[1:-1]
        else:
            parameters[lsplit[0].strip()] = float(lsplit[1])
        l += 1

    #Initializes the geometry dictionary.
    geometry = {}
    #1. ggxx(pi1,pj1,k) 
    geometry['ggxx'] = np.empty(0)
    #2. ggxy(pi1,pj1,k)
    geometry['ggxy'] = np.empty(0)
    #3. ggxz(pi1,pj1,k)
    geometry['ggxz'] = np.empty(0)
    #4. ggyy(pi1,pj1,k) 
    geometry['ggyy'] = np.empty(0)
    #5. ggyz(pi1,pj1,k)
    geometry['ggyz'] = np.empty(0)
    #6. ggzz(pi1,pj1,k)
    geometry['ggzz'] = np.empty(0)
    #7. gBfield(pi1,pj1,k) 
    geometry['gBfield'] = np.empty(0)
    #8. gdBdx(pi1,pj1,k)
    geometry['gdBdx'] = np.empty(0)
    #9. gdBdy(pi1,pj1,k)
    geometry['gdBdy'] = np.empty(0)
    #10. gdBdz(pi1,pj1,k)
    geometry['gdBdz'] = np.empty(0)
    #11. gjacobian(pi1,pj1,k)
    geometry['gjacobian'] = np.empty(0)
    #12. gl_R(pi1,k)
    geometry['gl_R'] = np.empty(0)
    #13. gl_phi(pi1,k)
    geometry['gl_phi'] = np.empty(0)
    #14. gl_z(pi1,k)
    geometry['gl_z'] = np.empty(0)
    #15. gl_dxdR(pi1,k)
    geometry['gl_dxdR'] = np.empty(0)
    #16. gl_dxdZ(pi1,k)
    geometry['gl_dxdZ'] = np.empty(0)

    #Populates the geometry dictionary with geom_coefficients
    l += 1
    while file_lines[l]:
        line = file_lines[l].split()
        geometry['ggxx'] = np.append(geometry['ggxx'],float(line[0].strip()))
        geometry['ggxy'] = np.append(geometry['ggxy'],float(line[1].strip()))
        geometry['ggxz'] = np.append(geometry['ggxz'],float(line[2].strip()))
        geometry['ggyy'] = np.append(geometry['ggyy'],float(line[3].strip()))
        geometry['ggyz'] = np.append(geometry['ggyz'],float(line[4].strip()))
        geometry['ggzz'] = np.append(geometry['ggzz'],float(line[5].strip()))
        geometry['gBfield'] = np.append(geometry['gBfield'],float(line[6].strip()))
        geometry['gdBdx'] = np.append(geometry['gdBdx'],float(line[7].strip()))
        geometry['gdBdy'] = np.append(geometry['gdBdy'],float(line[8].strip()))
        geometry['gdBdz'] = np.append(geometry['gdBdz'],float(line[9].strip()))
        geometry['gjacobian'] = np.append(geometry['gjacobian'],float(line[10].strip()))
        geometry['gl_R'] = np.append(geometry['gl_R'],float(line[11].strip()))
        geometry['gl_phi'] = np.append(geometry['gl_phi'],float(line[12].strip()))
        geometry['gl_z'] = np.append(geometry['gl_z'],float(line[13].strip()))
        geometry['gl_dxdR'] = np.append(geometry['gl_dxdR'],float(line[14].strip()))
        geometry['gl_dxdZ'] = np.append(geometry['gl_dxdZ'],float(line[15].strip()))
        l += 1

    return parameters, geometry

In [24]:
def init_read_geometry_file(suffix, pars):
    """
    Imports parameters from the magnetic geometry file GENE and assign values.
    Values determined from a 'magn_geometry'_XXXX GENE file. 'magn_geometry' is dependent on the
    simulation. 
    """
    geom_type = pars['magn_geometry'][1:-1]
    geom_file = geom_type + suffix
    geom_pars, geom_coeff = read_geometry_local(geom_file)

    return geom_type, geom_pars, geom_coeff

def read_curv_coeff(geom_type, geom_coeff, plot = False):
    """
    Determnies the curvature coeffiicients from the magn_geometry file and assigns values to
    corresponding variables.
    """
    #Geometry coefficients.
    ggxx = geom_coeff['ggxx']
    ggxy = geom_coeff['ggxy']
    ggxz = geom_coeff['ggxz']
    ggyy = geom_coeff['ggyy']
    ggyz = geom_coeff['ggyz']
    ggzz = geom_coeff['ggzz']

    #Determinate calculations.
    gamma1 = ggxx * ggyy - ggxy ** 2
    gamma2 = ggxx * ggyz - ggxy * ggxz
    gamma3 = ggxy * ggyz - ggyy * ggxz

    #Gradients of the magnetic field.
    gdBdx = geom_coeff['gdBdx']
    gdBdy = geom_coeff['gdBdy']
    gdBdz = geom_coeff['gdBdz']
    gBfield = geom_coeff['gBfield']

    #Curvature calculations
    Kx = - gdBdy - gamma2 / gamma1 * gdBdz
    Ky = gdBdx - gamma3 / gamma1 * gdBdz

    if (geom_type == 's_alpha'):
        Kx = Kx / gBfield
        Ky = Ky / gBfield

    return Kx, Ky


def read_curv_coeff_h5(geom_h5, coord, geom_type, plot = False):
    """
    Determnies the curvature coeffiicients from the magn_geometry h5 file and assigns values to
    corresponding variables.
    """
    #Geometry coefficients.
    ggxx = geom_h5.get('/metric/g^xx')[()]
    ggxy = geom_h5.get('/metric/g^xy')[()]
    ggxz = geom_h5.get('/metric/g^xz')[()]
    ggyy = geom_h5.get('/metric/g^yy')[()]
    ggyz = geom_h5.get('/metric/g^yz')[()]
    ggzz = geom_h5.get('/metric/g^zz')[()]

    #Determinate calculations.
    gamma1 = ggxx * ggyy - ggxy ** 2
    gamma2 = ggxx * ggyz - ggxy * ggxz
    gamma3 = ggxy * ggyz - ggyy * ggxz

    #Gradients of the magnetic field.
    gBfield = geom_h5.get('Bfield_terms/Bfield')[()]
    gdBdx = geom_h5.get('Bfield_terms/dBdx')[()]
    gdBdy = geom_h5.get('Bfield_terms/dBdy')[()]
    gdBdz = geom_h5.get('Bfield_terms/dBdz')[()]

    #Curvature calculations
    Kx = - gdBdy - gamma2 / gamma1 * gdBdz
    Ky = gdBdx - gamma3 / gamma1 * gdBdz

    if (geom_type == 's_alpha'):
        Kx = Kx / gBfield
        Ky = Ky / gBfield
        
    return Kx, Ky

In [25]:
def reconstruct_zgrid_h5(geom_h5, coord, pars, center_only = False):
    """
    Imports variables from input files to reconstruct the magnetic field space coordinates, the z grid.
    Input files are parameters_XXXX, coord_XXXX, and magn_geometry_XXXX. 
    """
    #Gets the number of grid points from parameters file. 
    nx = int(pars['nx0'])
    nz = int(pars['nz0'])

    #Initializes arrays for grid points and the jacobian. 
    zgrid = np.zeros(nx * nz, dtype = 'float128')
    jacobian = np.zeros(nx * nz, dtype = 'float128')

    #Gets coordinates, magnetic field jacobian, and the magnetic field. 
    zgrid_center = coord.get('/coord/z')[()]
    gjacobian = np.array(geom_h5.get('Bfield_terms/Jacobian')[()])
    Bfield = np.array(geom_h5.get('Bfield_terms/Bfield')[()])

    #Calculates zgrid, and jacobian for center_only coordinates.
    if center_only:
        ikx_grid = [0]
    else:
        ikx_grid = np.arange(- nx // 2 + 1, nx // 2 + 1)

    for i in ikx_grid:
        this_zgrid = i * 2. * np.pi + zgrid_center
        zgrid[(i - ikx_grid[0]) * nz: (i - ikx_grid[0] + 1) * nz] \
                      = this_zgrid
    if center_only:
        zgrid = zgrid_center

    jacobian_center = 1./gjacobian/Bfield

    #Calculates zgrid, and jacobian if not center_only coordinates.
    if not center_only:
        for i in ikx_grid:
            jacobian[(i-ikx_grid[0])*nz:(i-ikx_grid[0]+1)*nz]=jacobian_center
    if center_only:
        jacobian = jacobian_center

    return zgrid, jacobian